The dataset we are going to use today is **Boston Dataset**. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model
import random
from sklearn import feature_selection
import statsmodels.formula.api as smf
from sklearn import metrics
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing



In [2]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/Boston.csv"
BostonData = pd.read_csv(url)
del BostonData['Unnamed: 0']
BostonData.head(2)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.9,4.98,24.0
1,0.02731,0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6


The Boston data frame has 506 rows and 14 columns.
Usage

Boston

Format

This data frame contains the following columns:

crim

    per capita crime rate by town 
    
zn

    proportion of residential land zoned for lots over 25,000 sq.ft. 
    
indus

    proportion of non-retail business acres per town 
    
chas

    Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) 
    
nox

    nitrogen oxides concentration (parts per 10 million) 
    
rm

    average number of rooms per dwelling 
    
age

    proportion of owner-occupied units built prior to 1940 
    
dis

    weighted mean of distances to five Boston employment centres 
    
rad

    index of accessibility to radial highways 
    
tax

    full-value property-tax rate per 10,000 dollars
    
ptratio

    pupil-teacher ratio by town 
    
black

    1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town 
    
lstat

    lower status of the population (percent) 
    
medv

    median value of owner-occupied homes in 1000 dollars

Source

Harrison, D. and Rubinfeld, D.L. (1978) Hedonic prices and the demand for clean air. J. Environ. Economics and Management 5, 81–102.

Belsley D.A., Kuh, E. and Welsch, R.E. (1980) Regression Diagnostics. Identifying Influential Data and Sources of Collinearity. New York: Wiley.
[Package MASS version 7.2-29 Index]


#### Our goal is to predict the median value of properties (medv) based on other variables in this dataset.

#### First let's draw a scatter-plot of medv and lstat. Intuitively, does it like a pure linear association or it seems like there is some sort of non-linearity?

In [ ]:
BostonData.plot(kind="scatter", x="medv", y='lstat')

Answer: This looks like a curve based association, close to a line but not quite exactly on it.

#### Now, let's first define few non-linear terms. Start from a pure linear function and go up to polynomial degree 5. 

In [ ]:
BostonData['lstat_2'] = BostonData['lstat']**2
BostonData['lstat_3'] = BostonData['lstat']**3
BostonData['lstat_4'] = BostonData['lstat']**4
BostonData['lstat_5'] = BostonData['lstat']**5
X1 = BostonData[['lstat']]
X2 = BostonData[['lstat','lstat_2']]
X3 = BostonData[['lstat','lstat_2','lstat_3']]
X4 = BostonData[['lstat','lstat_2','lstat_3','lstat_4']]
X5 = BostonData[['lstat','lstat_2','lstat_3','lstat_4','lstat_5']]
y = BostonData['medv']


#### Now divide your dataset into 25% test set and 75% training set and use Validation and MSE of test set to decide which degree of polynomial fits the best. Run this procedure a few times!

In [ ]:
y = BostonData["medv"]

def get_min_index(scores):
    if len(scores) < 1:
        return -1 
    min_score = scores[0]
    min_index = 0
    for index, score in enumerate(scores):
        if score < min_score:
            min_index = index
            min_score = score
    return min_index

scores = []
lm = linear_model.LinearRegression()
list_count = []
index = np.array(range(5))+ 1

test_x = np.zeros(5)
train_x = np.zeros(5)

for num in range(9):
    for i,x in enumerate([X1, X2, X3, X4, X5]):
            X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25)
            lm.fit(X_train, Y_train)
            test_score = metrics.mean_squared_error(lm.predict(X_test), Y_test)
            train_score = metrics.mean_squared_error(lm.predict(X_train), Y_train)
            scores.append(test_score)
            test_x[i] = (test_score)
            train_x[i] = train_score
        #Calculate minimum_score 
        #print scores
    list_count.append(get_min_index(scores))
    scores = []
    MSE_Test_df = pd.DataFrame({'test_x':test_x,'index':index})
    MSE_Test_df.plot(x = 'index',y= 'test_x')

print Counter(list_count).most_common(3)

    


Answer: it seems validation is not a good method to handle this 

#### Now, on the same data set, use 10 fold cross-validation to decide on the degree of polynomial. Justify what you find.

In [ ]:
list_count = []
#for i in range(10):
kf = cross_validation.KFold(len(BostonData), n_folds = 10, shuffle=True)

test_x = np.zeros(5)
scores = []
for i, x in enumerate([X1, X2, X3, X4, X5]):
    for train_index, test_index in kf:
        lm.fit(x.iloc[train_index], y.iloc[train_index])
        err_test = lm.predict(x.iloc[test_index])
        current_score = metrics.mean_squared_error(err_test, y.iloc[test_index])
        scores.append(current_score)
    test_x[i] = np.mean(scores)
    #Calculate minimum_score 
        #print scores
    #list_count.append(get_min_index(scores))
    scores = []

MSE_Test_df = pd.DataFrame({'test_x':test_x,'index':index})
MSE_Test_df.plot(x = 'index',y= 'test_x')
#print Counter(list_count).most_common(3)



Answer: IT is much more overwhelmingly the 4th index in this case, degree 2 in this case is very useful as increasing polynomial degree can come with great cost.

# Now let's consider more variables.

#### Let's first focus on correlation Matrix.

In [ ]:
# Let's first get rid of additional variables we added to our dataframe
del BostonData['lstat_2']
del BostonData['lstat_3']
del BostonData['lstat_4']
del BostonData['lstat_5']

In [ ]:
BostonData.corr()

#### List 3 variables that have the highest chance to appear in your final model - the model that can predict medv. Can these variables appear simultaneously in your final model if your goal is interpretation?

Answer: the main 3 variables will be rm, lstat and ptratio. These variables cannot appear if our goal is interpretation as they are highly correlated.

#### Now let's standardize our data and put it in a new DataFrame called BostonDataNew

In [3]:
BostonDataNew = preprocessing.scale(BostonData)
BostonDataNew = pd.DataFrame(BostonDataNew)
BostonDataNew.columns = BostonData.columns.values

#### Now let's use 10-fold cross validation and Lasso regression on our standardized data to decide which variables to eliminate.

In [4]:
listOfAllVariables = BostonData.columns.values
X = BostonDataNew[listOfAllVariables]
Y = BostonDataNew["medv"]
del X["medv"]
X.head()



,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501


In [ ]:
kf = cross_validation.KFold(len(BostonDataNew), n_folds = 10, shuffle = True)
alphas = np.logspace(-10, 10, 21)
alpha_index = np.linspace(-10, 10, 21)
scores = []
MSE_lasso_csv = []
for a in alphas:
    print "Alpha: " , a
    scores = []
    for train_index, test_index in kf:
        lm = linear_model.Lasso(alpha=a).fit(X.iloc[train_index], Y.iloc[train_index])
        scores.append(metrics.mean_squared_error(lm.predict(X.iloc[test_index]), Y.iloc[test_index]))
    MSE_lasso_csv.append(np.mean(scores))

MSE_lasso_csv_df = pd.DataFrame({"MSE_lasso_csv" : MSE_lasso_csv, "Log_Alphas" : alpha_index})
MSE_lasso_csv_df.plot(x ="Log_Alphas", y = "MSE_lasso_csv")
    

In [7]:
lm = linear_model.Lasso(alpha=10**(-2)) #we could also use -2 but that doesn't give us a clear story.
lm.fit(X, Y)
print zip(lm.coef_,X.columns)

#Remove age, indus

[(-0.071461275127387022, 'crim'), (0.080283910755197502, 'zn'), (-0.0, 'indus'), (0.071857996341448307, 'chas'), (-0.1752046368173589, 'nox'), (0.30621113329030625, 'rm'), (-0.0, 'age'), (-0.26996579836071716, 'dis'), (0.14261427135427748, 'rad'), (-0.10216247651642624, 'tax'), (-0.2103399918606835, 'ptratio'), (0.083704310613461799, 'black'), (-0.40556056135991536, 'lstat')]


Answer: Remove age and indus from model

#### Now let's use 10-fold cross validation to choose our best model among the following candidates. Let's first add lstat**2 to our model. 

In [ ]:
BostonData['lstat_2'] = BostonData['lstat']**2
X1 = BostonData[['lstat']]
X2 = BostonData[['lstat','lstat_2']]
X3 = BostonData[['lstat','chas']]
X4 = BostonData[['lstat','lstat_2','chas']] #'black' is highly correlated with lstat so cannot consider them simoltanously
X5 = BostonData[['ptratio','chas']]
X6 = BostonData[['ptratio','chas','black']]
X7 = BostonData[['ptratio','black']]
X8 = BostonData[['rm']]
X9 = BostonData[['rm','chas']]
X10 = BostonData[['rm','chas','black']]
X11 = BostonData[['rm','black']]
X12 = BostonData[['lstat','ptratio','rm']]  #model without that much interpretability
X13 = BostonData[['lstat','lstat_2','ptratio','rm']]  #model without that much interpretability
X14 = BostonData[['lstat','ptratio','rm','chas','black']]  #model without that much interpretability
X15 = BostonData[['lstat','lstat_2','ptratio','rm','chas','black']]  #model without that much interpretability
y = BostonData['medv']

In [ ]:
# Use 10 fold cross-validation to decide on the model of your interest
kf = cross_validation.KFold(len(BostonData), n_folds = 10, shuffle=True)
overall_scores = []
for x in [X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15]:
    list_count = []
    for train_index, test_index in kf:
        lm.fit(x.iloc[train_index], y.iloc[train_index])
        err_test = lm.predict(x.iloc[test_index])
        current_score = metrics.mean_squared_error(err_test, y.iloc[test_index])
        scores.append(current_score)
    overall_scores.append({str(x.columns.values) : np.mean(scores)})
    scores = []
print overall_scores
    

#### If your goal is interpretation - what model(s) are you going to use? Use  smf.ols  in "statsmodels.formula.api as smf" to test significancy of your coefficients. 

Answer: 

In [ ]:

for x in [X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15]:
    lm1 = smf.ols(formula='y ~ x', data=BostonData).fit()
    #print(lm1.summary())
    print x.columns.values
    print(lm1.summary())


#### If your goal is prediction - what model(s) are you going to use? Use  smf.ols  in "statsmodels.formula.api as smf" to test significancy of your coefficients. 

Answer: 

In [ ]:
for i in [X12,X12,X14,X15]:
    lm1 = smf.ols(formula='y ~ i', data=BostonData).fit()
    print(lm1.summary())
    
    #All of our models are highly significant, so we use model 15. It generates the least CV-MSE. 